In [26]:
#primero importamos todos las bibliotecas necesarias
#importamos pytorch
import torch
#importamos el paquete de redes neuronales
import torch.nn as nn
import torch.nn.functional as F
#importamos el paquete que permite cargar los datos
from torch.utils.data import Dataset, Subset
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
#importamos el paquete que permite el autoentrenamiento del modelo
from torchvision.transforms import ToTensor
#importamos los dataloaders
from torch.utils.data import DataLoader
#importamos la función de optimización
from torch import optim
#importamos el paquete para el entrenamiento de la red
from torch.autograd import Variable

In [11]:
#establecemos el dispositivo a usar (cpu o gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
#antes de cargar los datos debemos definir un transformador que nos permita convertir las imágenes de formato PIL a formato Tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [39]:
#cargamos las imágenes del directorio que emplearemos para el entrenamiento
OCT = ImageFolder(root='Datos/Processed Data/OCT', transform = transforms)

In [40]:
#realizamos la división de las imágenes de entrenamiento en un 80% para train y un 20% para validation
n = len(OCT)
n_val = int(0.2*n)
OCT_train = Subset(OCT, range(n_val,n))
OCT_val = Subset(OCT, range(n_val))

In [41]:
#comprobamos que las dimensiones del conjunto de train y validation son las adecuadas
print(f'Longitud del conjunto de train: {len(OCT_train)}')
print(f'Longitud del conjunto de validation: {len(OCT_val)}')

Longitud del conjunto de train: 91
Longitud del conjunto de validation: 22


In [42]:
#definimos los dos conjuntos de test
Samsung_test = ImageFolder(root='Datos/Processed Data/Samsung', transform = transforms)
iPhone_test = ImageFolder(root='Datos/Processed Data/iPhone', transform = transforms)

In [43]:
#y observamos sus dimensiones
print(f'Longitud del conjunto de test de Samsung: {len(Samsung_test)}')
print(f'Longitud del conjunto de test de iPhone: {len(iPhone_test)}')

Longitud del conjunto de test de Samsung: 93
Longitud del conjunto de test de iPhone: 99


In [62]:
#a continuación debemos definir las funciones de carga de las imágenes
#vamos a establecer un tamaño de batch de 16 (ya que son pocas imágenes)
train_loader = DataLoader(dataset = OCT_train, batch_size = 16, shuffle = True, num_workers = 2)#genera subprocesos para cargar los datos y así liberamos el proceso main

val_loader = DataLoader(dataset = OCT_val, batch_size = 16, shuffle = True, num_workers = 2)

test_iPhone = DataLoader(dataset = iPhone_test, batch_size = 16, shuffle = True, num_workers = 2)
                         
test_Samsung = DataLoader(dataset = Samsung_test, batch_size = 16, shuffle = True, num_workers = 2)

In [54]:
#definimos el conjunto de etiquetas
classes = OCT.classes

In [55]:
#a continuación definimos la clase que contiene las capas, la arquitectura, del modelo
#vamos a comenzar con una arquitectura básica
#nuestra red hereda los atributos y funciones de la clase nn.Module
class CNN(nn.Module):
    def __init__(self):
        #extiende la función init de la clase nn.Module
        super(CNN, self).__init__()
        
        #definimos la primera capa, formada por una capa convolucional 2D, una ReLU y una maxpool
        self.conv1 = nn.Sequential(
            #capa convolucional
            nn.Conv2d(
                in_channels = 3,
                out_channels = 16, #se trata del número de salidas de la capa. Puede tratarse de un valor arbitrario
                kernel_size = 5, #suele tratarse de un número impar
                stride = 1, #cantidad píxeles que se desplaza el filtro sobre la imagen
                padding = 0, #cantidad de relleno que se va a aplicar sobre los bordes de la imagen
            ),
            
            #capa ReLU, simplifica los datos. Interpreta los valores positivos como son, y los negativos los torna 0
            #permite acelerar el entrenamiento
            nn.ReLU(),
            
            #capa de MaxPooling, reduce las dimensiones de los datos, seleccionando el valor máximo del kernel.
            nn.MaxPool2d(kernel_size=2)#cuanto mayor tamaño de kernel, mayor simplificación de los datos
        )
        
        #definimos la segunda capa, formada por los mismos componentes
        self.conv2 = nn.Sequential(
            #capa convolucional
            nn.Conv2d(
                in_channels = 16,
                out_channels = 32,
                kernel_size = 5,
                stride = 1,
                padding = 0
            ),
            
            #capa de ReLU
            nn.ReLU(),
            
            #capa de MaxPooling
            nn.MaxPool2d(kernel_size = 2)
        )
        
        #tras las 2 capas (que en verdad son 6) definimos una fully-connected, que es una red neuronal que en este caso tendrá 5 salidas (pues tenemos 5 posibles clases)
        self.out = nn.Linear(32*7*7,5)
        
        #definimos la función forward, que es la que llamará a las capas creadas y obtendrá las predicciones.
        #recibe como parámetro el batch de imágenes
        def forward(self,x):
            #primero las dos estructuras convolucionales
            x = self.conv1(x)
            x = self.conv2(x)
            #después aplanamos la salida, hasta convertirla de forma matricial a forma vectorial (sería la capa flatten)
            x = x.view(x.size(0),-1)
            #y por último la salida, la predicción
            output = self.out(x)
            #no incluimos una capa de LogSoft, que convierte el output en probabilidad, ya que la función loss que usaremos incluye esta funcionalidad
            return output, x

In [56]:
#una vez definida la clase vamos a crear la instancia y visualizar su contenido
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=5, bias=True)
)


In [57]:
#a continuación definimos la función que permite calcular el loss y el optimizador
loss = nn.CrossEntropyLoss()

optimizer = optim.Adam(cnn.parameters(), lr = 0.01)

In [49]:
#por último debemos entrenar la red creada y obtener sus métricas de rendimiento. 
#Para ello definimos una función que ejecuta el entrenamiento de 1 época


In [71]:
for i,j in enumerate(train_loader):
    print(i)

TypeError: cannot pickle 'module' object